In [1]:
import numpy as np
import pandas as pd

import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

from sklearn.preprocessing import StandardScaler
import pickle


In [2]:
dataset = pd.read_csv(r"npk_new.csv")
dataset

,N,P,K,LABEL
0,80,77,73,Beans
1,63,73,74,Beans
2,85,61,78,Beans
3,116,59,54,Beans
4,96,64,69,Beans
...,...,...,...,...
445,118,47,77,Watermelon
446,157,57,66,Watermelon
447,144,43,77,Watermelon
448,148,47,61,Watermelon


In [3]:
labels = dataset['LABEL'].unique().tolist()
labels.sort()
labels

['Beans',
 'Brinjal',
 'Cabbage',
 'Carrot',
 'Cauliflower',
 'Chilli',
 'Cowpea',
 'Cucumber',
 'Garden pea',
 'Garlic',
 'Lettuce',
 'Okra',
 'Onion',
 'Potato',
 'Snap bean',
 'Spinach',
 'Tomato',
 'Watermelon']

In [4]:
le = LabelEncoder()
le.fit(dataset['LABEL'])
dataset['LABEL']=le.transform(dataset['LABEL'])

In [5]:
dataset

,N,P,K,LABEL
0,80,77,73,0
1,63,73,74,0
2,85,61,78,0
3,116,59,54,0
4,96,64,69,0
...,...,...,...,...
445,118,47,77,17
446,157,57,66,17
447,144,43,77,17
448,148,47,61,17


In [6]:
scaler = StandardScaler()
#scaler.fit(X)
dataset.iloc[0:,:-1] =scaler.fit_transform(dataset.iloc[0:,:-1]) 

In [7]:

# selecting the features and labels
X = dataset.iloc[:, :-1].values
Y = dataset.loc[:, ['LABEL']]

print(X)
print(Y)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# splitting the model into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=100)


[[-1.20811989  0.85803423  0.20411868]
 [-1.59507841  0.56010568  0.26605625]
 [-1.09430857 -0.33367998  0.5138065 ]
 ...
 [ 0.24866511 -1.67435847  0.45186894]
 [ 0.33971417 -1.37642992 -0.53913208]
 [-0.54801419 -0.5571264  -0.53913208]]
     LABEL
0        0
1        0
2        0
3        0
4        0
..     ...
445     17
446     17
447     17
448     17
449     17

[450 rows x 1 columns]


In [8]:

# # training a logistics regression model
# logmodel = LogisticRegression()
# logmodel.fit(X_train, y_train.values.ravel())
# predictions = logmodel.predict(X_test)
# print("***********************************************************")
# print("Accuracy = " + str(accuracy_score(y_test.values.ravel(), predictions)))
# print("***********************************************************")

sgd_classifier = SGDClassifier(tol=-np.infty, loss='log')
sgd_classifier.fit(X_train, y_train)

predictions = sgd_classifier.predict(X_test)
print("***********************************************************")
print("Accuracy = " + str(accuracy_score(y_test.values.ravel(), predictions)))
print("***********************************************************")

C:\Users\PRAMILA\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


***********************************************************
Accuracy = 0.3893805309734513
***********************************************************


In [9]:
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X_train, y_train.values.ravel())
predictions = regressor.predict(X_train)
print("***********************************************************")

print(regressor.score(X_train, y_train))

print("***********************************************************")

***********************************************************
0.8924987695074404
***********************************************************


In [10]:
pickle.dump(regressor, open('randomforest_model.pkl','wb'))

In [11]:
test_data=[[0,0,144]]
print(regressor.predict([[0,42,144]]))

# Loading model to compare the results
svm_model = pickle.load(open('randomforest_model.pkl','rb'))
o=svm_model.predict(test_data)
print(o)
print(labels[int(o[0])])


[3.97]
[11.54]
Okra


In [12]:
results = pd.DataFrame(sgd_classifier.predict_proba([[137,42,144]]).transpose())
results.rename(columns={0: "score"}, inplace=True)

results.reset_index(level=0, inplace=True)
results.sort_values(by="score", ascending=False, inplace=True, ignore_index=True)



In [13]:
for i in range(len(results)):
    if results.loc[i, "score"] > 0.001:
        print(str(labels[results.loc[i, "index"]]) + " (" + str(round(100 * results.loc[i, "score"])) + "%)")



Watermelon (10%)
Brinjal (10%)
Carrot (10%)
Chilli (10%)
Potato (10%)
Onion (10%)
Lettuce (10%)
Tomato (10%)
Cauliflower (10%)
Garlic (10%)
